In [1]:
import re
import requests
import urllib
from bs4 import BeautifulSoup
import random
import pandas as pd
import csv
from itertools import cycle
import time
import numpy as np

In [2]:
#获取的数据包括：
#1.标题 2.作者昵称 3.作者id 4.回应数 
#5.时间 6.帖子内容 7. 帖子url 8.回帖内容 9.帖子中包含图片的数量

In [3]:
HEADERS_LIST = [
    'Mozilla/5.0 (Windows; U; Windows NT 6.1; x64; fr; rv:1.9.2.13) Gecko/20101203 Firebird/3.6.13',
    'Mozilla/5.0 (compatible, MSIE 11, Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201',
    'Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16',
    'Mozilla/5.0 (Windows NT 5.2; RW; rv:7.0a1) Gecko/20091211 SeaMonkey/9.23a1pre'
]

In [4]:
#随机更换代理，是反爬的trick之一，但是在实践过程中，并不work；
#更换header也是，但是从爬取的实践经验来看，time.sleep()的设置是非常重要的；
#sleep的时间设置长些，也就意味着访问url的间隔时间会长些，这样可以有效避免反爬；
def get_proxies():
    response = requests.get(PROXY_URL)
    soup = BeautifulSoup(response.text, 'lxml')
    table = soup.find('table',id='proxylisttable')
    list_tr = table.find_all('tr')
    list_td = [elem.find_all('td') for elem in list_tr]
    list_td = list(filter(None, list_td))
    list_ip = [elem[0].text for elem in list_td]
    list_ports = [elem[1].text for elem in list_td]
    list_proxies = [':'.join(elem) for elem in list(zip(list_ip, list_ports))]
    return list_proxies  

def get_data(num, HEADER):
    basic_url = 'https://www.douban.com/group/638298/discussion?start=%d'%(num*25)
    #basic_url = 'https://www.douban.com/group/638298/discussion?start=25'
    response = requests.get(basic_url, headers = HEADER, timeout= 60)
    html = response.text
    soup = BeautifulSoup(html, 'lxml')
    
    #获取帖子中的title,name,response,time;
    #filtered_data = ['title','name','response','time']
    data = soup.find_all('td')
    filtered_data = [str(i.text.strip()) for i in data][5:]#['','']
        
    #获取帖子中的topic's url及发帖人id；
    topic_people_urls = [url.get('href') for url in soup.find_all('a')]
    topic_url = []
    people_url = []
    for topic_people_url in topic_people_urls:
        topic_people_url_ = topic_people_url.split('/')
        #获得topic_url;
        if len(topic_people_url_) < 3:
            continue
        if topic_people_url_[-3] == 'topic':
            topic_url.append(topic_people_url)
        #获得people_url;
        elif topic_people_url_[-3] == 'people':
            people_url.append(topic_people_url)
             
    return filtered_data, topic_url, people_url

def get_content(topic_urls, HEADER):
    
    docs = []
    count_ = []
    reply_content = []
    img_urls_ = []
    for url in topic_urls:
        response = requests.get(url, headers = HEADER, timeout= 60)
        html = response.text
        soup = BeautifulSoup(html, 'lxml')
        
        #获取帖子内容；
        titles = soup.find_all('div','topic-doc')
        topic_docs = [str(i.text.strip()) for i in titles]
        try:
            topic_docs_ = [i.split('\n\n\n\n\n')[1].strip() for i in topic_docs]
        except:
            topic_docs_ = ['空']
        docs.extend(topic_docs_)
        
        #获取帖子中含有图片的数量；
        img_urls = [i.get('src') for i in soup.find_all('img')]
        img_urls_.append(img_urls)
        
        #获取回帖内容；
        reply = [i.text.strip() for i in soup.find_all('div', 'reply-doc content')]
        reply_ = [i.split('\n\n')[1].strip() for i in reply]
        reply_content.append(reply_)
        time.sleep(3)
        
    #获取帖子所发图片的数量
    for i in img_urls_:
        count = 0
        for j in i:
            if j.split('/')[-2] == 'public':
                count = count + 1
        count_.append(count)

    return docs, reply_content, count_

def get_page_nums():
    basic_url = 'https://www.douban.com/group/638298/discussion?start=0'
    response = requests.get(basic_url, headers = HEADER)
    html = response.text
    soup = BeautifulSoup(html, 'lxml')
    nums = str(soup.find('span', 'thispage')).split('"')[-2]
    return nums

In [7]:
HEADER = {'User-Agent': random.choice(HEADERS_LIST)}
nums = get_page_nums()

#for i in range(int(nums)+1):
for i in range(1):
    s_time = time.time()
    print(i)
    HEADER = {'User-Agent': random.choice(HEADERS_LIST)}
    
    try:
        filtered_data, topic_url, people_url = get_data(i, HEADER)
        time.sleep(3)
        docs, reply_content, count_= get_content(topic_url, HEADER)
    except:
        filtered_data, topic_url, people_url = get_data(i, HEADER)
        time.sleep(3)
        docs, reply_content, count_= get_content(topic_url, HEADER)
        
    titles = [filtered_data[i] for i in range(0, len(filtered_data), 4)]
    names = [filtered_data[i] for i in range(1, len(filtered_data), 4)]
    responses = [filtered_data[i] for i in range(2, len(filtered_data), 4)]
    times = [filtered_data[i] for i in range(3, len(filtered_data), 4)]
    
    csv = pd.DataFrame({'title':titles, 'name':names, 'id':people_url,
                        'response_nums':responses, 'time':times, 'docs':docs,
                        'reply_content':reply_content, 'img_nums':count_, 
                       'topic_url':topic_url})
    csv.to_csv(r'hhhhhhhh.csv', mode= 'a', header= False)
    time.sleep(5)
    print('哈哈哈哈哈哈哈')
    e_time = time.time()
    print(e_time-s_time)
    print('===========')

0
哈哈哈哈哈哈哈
126.90873217582703
